## A Journey into bioinformatics
Original tutorial I followed while developing this notebook: <a href="https://www.youtube.com/watch?v=jBlTQjcKuaY">Python for Bioinformatics - Drug Discovery Using Machine Learning and Data Analysis</a><br>
## Data collection - <a href="https://www.ebi.ac.uk/chembl/">ChEMBL Database</a>
The ChEMBL Database is a database containing curated bioactivity data for more than 2 million compounds and was compiled with over 76,000 documents, 1.2 million essays and the data spans 13,000 targets and 1,800 cells and 33,000 indications (version 26)<br>
### Import section

In [1]:
# You may need to: pip install chembl_webresource_client

import pandas as pd 
import os

import plotly.express as px
from plotly.subplots import make_subplots

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold

from numpy.random import seed
from numpy.random import randn
from scipy.stats import mannwhitneyu

from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

from chembl_webresource_client.new_client import new_client

from lazypredict.Supervised import LazyRegressor

C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


### Search for a target protein
Acetylcholinesterase in this case. Specifically the single protein target_type for the Homo sapien organism

In [2]:
qry = new_client.target.search("acetylcholinesterase")
dfsearch = pd.DataFrame(qry)
dfsearch[(dfsearch['organism']=='Homo sapiens') & (dfsearch['target_type']=='SINGLE PROTEIN')]

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,"[{'xref_id': 'P22303', 'xref_name': None, 'xre...",Homo sapiens,Acetylcholinesterase,27.00,False,CHEMBL220,"[{'accession': 'P22303', 'component_descriptio...",SINGLE PROTEIN,9606


Now we will aquire the bioactivity for Human Acetylcholinesterase that are reported as pChEMBL values

In [3]:
chembl_id = dfsearch['target_chembl_id'][0]
csv_name = "bioactivity_"+chembl_id +"_raw.csv"

# If we don't already have the file, then download it
if not os.path.exists(csv_name):
    activity = new_client.activity
    res = activity.filter(target_chembl_id=chembl_id).filter(standard_type="IC50")
    df = pd.DataFrame(res)
    df.to_csv(csv_name, index=False)
else:
    df = pd.read_csv(csv_name)
df.head()

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,NaN,33969,[],CHEMBL643384,Inhibitory concentration against acetylcholine...,B,nan,nan,BAO_0000190,BAO_0000357,...,Homo sapiens,Acetylcholinesterase,9606,nan,nan,IC50,uM,UO_0000065,nan,0.75
1,NaN,37563,[],CHEMBL643384,Inhibitory concentration against acetylcholine...,B,nan,nan,BAO_0000190,BAO_0000357,...,Homo sapiens,Acetylcholinesterase,9606,nan,nan,IC50,uM,UO_0000065,nan,0.10
2,NaN,37565,[],CHEMBL643384,Inhibitory concentration against acetylcholine...,B,nan,nan,BAO_0000190,BAO_0000357,...,Homo sapiens,Acetylcholinesterase,9606,nan,nan,IC50,uM,UO_0000065,nan,50.00
3,NaN,38902,[],CHEMBL643384,Inhibitory concentration against acetylcholine...,B,nan,nan,BAO_0000190,BAO_0000357,...,Homo sapiens,Acetylcholinesterase,9606,nan,nan,IC50,uM,UO_0000065,nan,0.30
4,NaN,41170,[],CHEMBL643384,Inhibitory concentration against acetylcholine...,B,nan,nan,BAO_0000190,BAO_0000357,...,Homo sapiens,Acetylcholinesterase,9606,nan,nan,IC50,uM,UO_0000065,nan,0.80


### Wrangling section
<ul><li>Keep only the necessary columns</li>
<li>Drop nulls</li>
<li>Drop dupes</li>
<li>Add a "Class" feature</li>
</ul>

In [4]:
def classify_standard_value(x):
    ''' INPUTS: x: the standard value of the molecule
        OUTPUTS: the "class" as per the standard value range'''
    x = float(x)
    if x >= 10000:
        return "inactive"
    elif x > 1000:
        return "intermediate"
    return "active"

def clean_smile(x):
    cpd = str(x).split('.')
    return max(cpd,key=len)

if not os.path.exists("bioactivity_"+chembl_id +"_clean.csv"):
    dfc = df[["molecule_chembl_id","canonical_smiles","standard_value"]].copy()
    dfc.dropna(axis=0,how="any", subset=["standard_value","canonical_smiles"], inplace=True)
    dfc.drop_duplicates(["canonical_smiles"], inplace=True)
    dfc['class'] = dfc['standard_value'].apply(lambda x: classify_standard_value(x))
    dfc['canonical_smiles'] = dfc['canonical_smiles'].apply(lambda x: clean_smile(x))
    dfc.to_csv("bioactivity_"+chembl_id +"_clean.csv", index=False)
else:
    dfc = pd.read_csv("bioactivity_"+chembl_id +"_clean.csv")
dfc.head()

,molecule_chembl_id,canonical_smiles,standard_value,class
0,CHEMBL133897,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,750.00,active
1,CHEMBL336398,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,100.00,active
2,CHEMBL131588,CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1,50000.00,inactive
3,CHEMBL130628,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F,300.00,active
4,CHEMBL130478,CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C,800.00,active


### Calculate Lipinski descriptors
Lipinski's "Rule of five" states:
<ul><li>Molecular weight < 5</li>
<li>Octanol-water partition coefficient (LogP) < 5</li>
<li>Hydrogen bond donors < 5</li>
<li>Hydrogen bond acceptors < 10</li>
</ul>

In [5]:
def get_lipinski_descriptors(smile):
    mol=Chem.MolFromSmiles(smile)

    MolWt=Descriptors.MolWt(mol)
    MolLogP=Descriptors.MolLogP(mol)
    NumHDonors = Lipinski.NumHDonors(mol)
    NumHAcceptors = Lipinski.NumHAcceptors(mol)
    return MolWt,MolLogP,NumHDonors,NumHAcceptors
dfc['MolWt'] = -1
dfc['MolLogP'] = -1
dfc['NumHDonors'] = -1
dfc['NumHAcceptors'] = -1

dfc["MolWt"],dfc["MolLogP"],dfc["NumHDonors"],dfc["NumHAcceptors"] = zip(*dfc['canonical_smiles'].map(get_lipinski_descriptors))
dfc.head()

,molecule_chembl_id,canonical_smiles,standard_value,class,MolWt,MolLogP,NumHDonors,NumHAcceptors
0,CHEMBL133897,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,750.00,active,312.33,2.80,0,6
1,CHEMBL336398,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,100.00,active,376.91,4.55,0,5
2,CHEMBL131588,CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1,50000.00,inactive,426.85,5.36,0,5
3,CHEMBL130628,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F,300.00,active,404.85,4.71,0,5
4,CHEMBL130478,CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C,800.00,active,346.33,3.10,0,6


### Convert IC50 to pIC50
Converting IC50 to pIC50 allows a more uniform distribution. This is a negative logarithmic scale

In [6]:
def convert_IC50(x):
    if x > 100000000:
        x = 100000000
    molar = x*(10**-9) #Convert nM to M
    return -np.log10(molar)

dfc['pic50'] = dfc['standard_value'].apply(lambda x: convert_IC50(x))
px.histogram(dfc,x="standard_value", title="IC50 distribution", height=300, width=300)

In [7]:
px.histogram(dfc,x="pic50", title="pIC50 distribution", height=300, width=300)

## Exploratory analysis

In [8]:
# Removing the Intermediate class from our analysis dataset
#dfc = dfc[dfc['class']!='intermediate'].copy()

fig = px.histogram(dfc[dfc['class']!='intermediate'], x="class", color="class", height=300, width=300, title="Class frequency")
fig.update_layout(showlegend=False)
fig.show()

In [9]:
px.scatter(dfc[dfc['class']!='intermediate'], x="MolWt", y="MolLogP", color="class", size="pic50", size_max=10
    , opacity=0.7, height=400, width=600, title="Molecular Weight v. LogP")

In [10]:
fig = px.box(dfc[dfc['class']!='intermediate'], x="class", y="pic50", color="class", title="pIC50 by class", height=400, width=400)
fig.update_layout(showlegend=False)
fig.show()

## Statistical analysis: Mann-Whitney U Test

In [11]:
def mannwhitney(df, descriptor, alpha=0.05, seed_num=42, verbose=False):
    seed(seed_num)

    active = df[df["class"]=="active"][descriptor]
    inactive = df[df["class"]=="inactive"][descriptor]
    stat, p = mannwhitneyu(active, inactive)

    interpretation = "Same distribution (fail to reject H0)"
    if p <= alpha:
        interpretation = "Difference distribution (reject H0)"

    return pd.DataFrame({"Descriptor":descriptor, "Statistics":stat, "P":p, "alpha":alpha,"Interpretation":interpretation}, index=[0])

def generate_mannwhitney_report(df, features):
    dfr = pd.DataFrame()
    for f in features:
        dfr = pd.concat([dfr,mannwhitney(df, f, alpha=0.01)])
    return dfr

def plot_boxes(df, x_col, y_cols=[]):
    fig = make_subplots(rows=1,cols=len(y_cols)
        , subplot_titles=features)
    x_len = len(df[x_col].unique())
    col = 1
    for y in y_cols:
        for i in range(x_len):
            fig.add_trace(px.box(df,x=x_col,y=y,title=y, color=x_col).data[i], row=1, col=col)
        col+=1

    fig.update_layout(height=400, width=300*len(y_cols), showlegend=False)
    return fig

In [12]:
features=["pic50","MolWt", "MolLogP", "NumHDonors", "NumHAcceptors"]
dfmw = generate_mannwhitney_report(dfc[dfc['class']!='intermediate'], features)
dfmw

,Descriptor,Statistics,P,alpha,Interpretation
0,pic50,0.00,0.00,0.01,Difference distribution (reject H0)
0,MolWt,1207234.00,0.00,0.01,Difference distribution (reject H0)
0,MolLogP,1176292.50,0.00,0.01,Difference distribution (reject H0)
0,NumHDonors,1524372.50,0.00,0.01,Difference distribution (reject H0)
0,NumHAcceptors,1636225.00,0.00,0.01,Difference distribution (reject H0)


In [13]:
plot_boxes(dfc[dfc['class']!='intermediate'],x_col="class",y_cols=features).show()

In [14]:
if not os.path.exists("descriptors_output.csv"): #Generate descriptors. This will take approx 0.21 seconds per descriptor
    dfc[["canonical_smiles","molecule_chembl_id"]].to_csv("molecule.smi", sep="\t", index=False,header=False)
    os.system("padel.sh")
    
dfd = pd.read_csv("descriptors_output.csv")
dfd = dfd.drop(columns="Name")
dfd.tail()

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
5038,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5039,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5040,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5041,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5042,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [15]:
if not os.path.exists("acetylcholinesterase_pubchemFP_pIC50.csv"):
    dfm = pd.concat([dfd,dfc['pic50']], axis=1)
    dfm.dropna(axis=0, inplace=True)
    dfm.to_csv("acetylcholinesterase_pubchemFP_pIC50.csv", index=False)
else:
    dfm = pd.read_csv("acetylcholinesterase_pubchemFP_pIC50.csv")

## Computational drug discovery
### Regression models with Random Forest
Train Test data split

In [16]:
y = dfm['pic50']
x = dfm.drop('pic50', axis=1)
x = VarianceThreshold(threshold=(.8*(1-.8))).fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
x_train.shape, y_train.shape, x_test.shape,y_test.shape

((4034, 140), (4034,), (1009, 140), (1009,))

### Build, then plot a Random Forest model

In [17]:
seed(42)
model = RandomForestRegressor(n_estimators=100)
model.fit(x_train, y_train)
model.score(x_test, y_test)

0.3755509721088499

In [18]:
y_pred = model.predict(x_test)
fig = px.scatter(x=y_test,y=y_pred, opacity=0.4, trendline="ols", height=400, width=400)
fig.update_layout(xaxis_title="Experimental pIC50", yaxis_title="Predicted pIC50")
fig.show()

## Compare multiple Supervised Models

In [46]:
models_test,predictions_test = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None).fit(x_train, x_test, y_train, y_test)

100%|██████████| 42/42 [01:17<00:00,  1.85s/it]


In [48]:
models_train,predictions_train = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None).fit(x_train, x_train, y_train, y_train)

100%|██████████| 42/42 [01:27<00:00,  2.09s/it]


In [49]:
predictions_train

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
DecisionTreeRegressor,0.81,0.82,0.69,0.20
ExtraTreeRegressor,0.81,0.82,0.69,0.20
ExtraTreesRegressor,0.81,0.82,0.69,9.87
GaussianProcessRegressor,0.81,0.82,0.69,9.77
RandomForestRegressor,0.76,0.77,0.77,6.74
XGBRegressor,0.75,0.76,0.78,1.31
BaggingRegressor,0.74,0.75,0.81,0.90
MLPRegressor,0.69,0.70,0.88,11.77
HistGradientBoostingRegressor,0.55,0.57,1.05,8.57


In [50]:
predictions_test

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
HistGradientBoostingRegressor,0.28,0.38,1.29,8.58
LGBMRegressor,0.28,0.38,1.29,0.35
RandomForestRegressor,0.27,0.38,1.30,6.83
BaggingRegressor,0.25,0.35,1.32,0.81
NuSVR,0.23,0.34,1.34,4.14
SVR,0.22,0.33,1.34,4.54
XGBRegressor,0.22,0.33,1.34,1.32
KNeighborsRegressor,0.19,0.30,1.37,1.43
GradientBoostingRegressor,0.19,0.30,1.37,2.59


In [51]:
dfPlt = predictions_train[predictions_train["R-Squared"] > -1].sort_values(["R-Squared"], ascending=True)
px.bar(dfPlt,y=dfPlt.index, x="R-Squared")